In [7]:
# Telco Churn Prediction with Tkinter GUI

import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox

# ML libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# ------------------------
# 1. Load and preprocess dataset
# ------------------------
dataset = pd.read_csv("Telco-Customer-Churn.csv")

# Convert TotalCharges to numeric
dataset['TotalCharges'] = pd.to_numeric(dataset['TotalCharges'], errors='coerce')
dataset['TotalCharges'] = dataset['TotalCharges'].fillna(dataset['TotalCharges'].median())

# Encode categorical columns
label_encoders = {}
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']

for col in categorical_cols:
    le = LabelEncoder()
    dataset[col] = le.fit_transform(dataset[col])
    label_encoders[col] = le  # store encoder for later use

# Features and target
X = dataset.drop(['customerID', 'Churn'], axis=1)
y = dataset['Churn']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


# ------------------------
# 2. Tkinter GUI
# ------------------------
def predict_churn():
    try:
        # Get inputs from GUI
        tenure = int(entry_tenure.get())
        monthly_charges = float(entry_monthly.get())
        total_charges = float(entry_total.get())
        gender = combo_gender.get()
        contract = combo_contract.get()

        # Build a DataFrame row with default values
        input_dict = {col: 0 for col in X.columns}
        input_dict['tenure'] = tenure
        input_dict['MonthlyCharges'] = monthly_charges
        input_dict['TotalCharges'] = total_charges
        input_dict['gender'] = label_encoders['gender'].transform([gender])[0]
        input_dict['Contract'] = label_encoders['Contract'].transform([contract])[0]

        input_df = pd.DataFrame([input_dict])

        # Scale input
        final_input = scaler.transform(input_df)

        # Predict
        prediction = model.predict(final_input)[0]
        prob = model.predict_proba(final_input)[0][1]

        result = "Churn" if prediction == 1 else "No Churn"
        messagebox.showinfo("Prediction Result", f"Prediction: {result}\nProbability: {prob:.2f}")

    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")


# ------------------------
# 3. Build GUI Layout
# ------------------------
root = tk.Tk()
root.title("Telco Customer Churn Prediction")
root.geometry("400x400")

# Labels and Entries
tk.Label(root, text="Tenure (months):").pack(pady=5)
entry_tenure = tk.Entry(root)
entry_tenure.pack()

tk.Label(root, text="Monthly Charges:").pack(pady=5)
entry_monthly = tk.Entry(root)
entry_monthly.pack()

tk.Label(root, text="Total Charges:").pack(pady=5)
entry_total = tk.Entry(root)
entry_total.pack()

tk.Label(root, text="Gender:").pack(pady=5)
combo_gender = ttk.Combobox(root, values=label_encoders['gender'].classes_.tolist())
combo_gender.pack()

tk.Label(root, text="Contract Type:").pack(pady=5)
combo_contract = ttk.Combobox(root, values=label_encoders['Contract'].classes_.tolist())
combo_contract.pack()

# Predict button
tk.Button(root, text="Predict Churn", command=predict_churn, bg="blue", fg="white").pack(pady=20)

root.mainloop()
